In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings)
import time

In [2]:
pbp2022=pd.read_csv(filepath_or_buffer="PBP Data/2021-22_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls last year

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2021-22",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2021-22",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame().set_index("PLAYER_ID")

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(5,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
37,Blake Griffin,1.136842
358,Kyle Lowry,0.640233
521,Taurean Prince,0.572185
417,Moritz Wagner,0.500870
17,Alperen Sengun,0.450000


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(5,columns='LOOSE_BALLS_RECOVERED',keep='all')\
[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
292,John Konchar,1.658182
381,Malachi Flynn,1.554545
479,Robert Williams III,1.462116
210,Hamidou Diallo,1.440000
580,Xavier Tillman,1.415172


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(5,columns='DEFLECTIONS',keep='all')\
[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
398,Matisse Thybulle,5.123077
195,Gary Payton II,5.121429
128,De'Anthony Melton,4.628571
13,Alex Caruso,4.551429
55,Bruce Brown,4.528000


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(5,columns='SCREEN_AST_PTS',keep='all')\
[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
487,Rudy Gobert,18.067290
244,Jakob Poeltl,16.269474
163,Drew Eubanks,16.015748
515,Steven Adams,15.820312
322,Jusuf Nurkic,15.459623


# The Deadshot Award (presented by Ray Allen/Reggie Miller)

best qualifying 3 point percentage (Basketball-Reference)

1. P.J. Tucker (46.9%)
2. Joe Harris (46.6%)
3. Eric Gordon (44.8%)
4. Tyrese Haliburton (44.5%)
5. P.J. Washington (44.3%)

# The Stormtrooper Award

worst qualifying 2 point percentage (Basketball-Reference)

1. Ricky Rubio (38%)
2. Devonte' Graham (40%)
3. Buddy Hield (41.2%)
4. Lonzo Ball (41.4%)
5. Nickeil Alexander-Walker & Saddiq Bey (42.2%)

# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2022[pbp2022.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2022.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).\
nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
26,Daniel Gafford,6
27,Darius Bazley,5
53,Ivica Zubac,5
76,Karl-Anthony Towns,5
106,Precious Achiuwa,5


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).\
nlargest(5,columns='count',keep='all')

,PLAYER3_NAME,count
6,Andre Drummond,8
25,Daniel Gafford,5
41,Evan Mobley,5
103,Myles Turner,5
7,Anthony Davis,4
20,Clint Capela,4
45,Giannis Antetokounmpo,4
61,Jaren Jackson Jr.,4
63,Jarrett Allen,4


# The "If He Dies, He Dies" Award (presented by Tom Thibodeau, sponsored by Ivan Drago)

most minutes played per game (Basketball-Reference) (credit to FurryCrew for the idea)

1. Fred VanVleet (37.5)
2. Kevin Durant (37.1)
3. James Harden & Jayson Tatum (36.6)
4. Damian Lillard (36.4)

alternatively: most total minutes played (Basketball-Reference) (credit to FrankEMartindale for the idea)

1. Russell Westbrook (1427)
2. Miles Bridges (1359)
3. Domantas Sabonis (1351)
4. Julius Randle (1350)
5. Mikal Bridges (1325)

# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)].\
drop(columns=['PLAYER3_ID','PLAYER3_NAME','PLAYER3_TEAM_ABBREVIATION'])
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).\
nlargest(5,columns='count',keep='all')

,p1,p2,count
125,Clint Capela,Trae Young,26
150,Darius Garland,Jarrett Allen,23
353,Mike Conley,Rudy Gobert,23
41,Anthony Davis,Russell Westbrook,16
304,John Collins,Trae Young,16
348,Marcus Smart,Robert Williams III,16


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)
hustle_50percent_gp_ranks.nsmallest(n=5,columns='sum',keep='all')\
[['PLAYER_NAME','CHARGES_DRAWN_pct_rank','CONTESTED_SHOTS_pct_rank','DEFLECTIONS_pct_rank',
'DEF_BOXOUTS_pct_rank','DEF_LOOSE_BALLS_RECOVERED_pct_rank','sum']]

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
22,Anfernee Simons,0.24234,0.181058,0.033426,0.018106,0.172702,0.647632
100,D.J. Augustin,0.24234,0.009749,0.050139,0.091922,0.317549,0.711699
524,Terrence Ross,0.24234,0.097493,0.495822,0.064067,0.008357,0.908078
575,Will Barton,0.24234,0.217270,0.374652,0.013928,0.164345,1.012535
286,Joe Ingles,0.24234,0.066852,0.178273,0.462396,0.072423,1.022284


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played

1. Luka Doncic (87.7%)
2. Shai Gilgeous-Alexander (86.8%)
3. Chris Paul (84.8%)
4. Trae Young (84.2%)
5. Kevin Pangos (80%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Juancho Hernangomez (100%)
2. Ben McLemore (96.3%)
3. Mike Muscala & Maxi Kleber (94.9%)
4. Wayne Ellington (94.8%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

1. Kevin Pangos (0%)
2. Justin Robinson (4.8%)
3. Jalen Green (6.7%)
4. Tyus Jones (6.8%)
5. Kira Lewis Jr. (7.7%)

alternatively: restricting to players > 6 foot 7 inches in height

1. Furkan Korkmaz (13.1%)
2. Amir Coffey (14.7%)
3. Timothe Luwawu-Cabarrot (15.4%)
4. Cedi Osman (16.4%)
5. Cam Reddish (16.7%)

# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

1. Jock Landale (70%)
2. Mitchell Robinson (60.1%)
3. Moses Moody (60%)
4. Luka Garza (58.1%)
5. Jakob Poeltl (57.2%)

alternatively: restricting to players < 6 foot 7 inches in height

1. Moses Moody (60%)
2. Justin Champagnie (50%)
3. Thanasis Antetokounmpo (46.8%)
4. Jared Butler (46.7%)
5. Eric Paschall (41.3%)

# The Rotation Awards

**value depth over one solitary star. used basketball-reference's play-by-play position percentages as initial baseline for position groupings**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)